In [1]:

!pip install snowflake-connector-python


In [13]:
!pip install "snowflake-connector-python[pandas]"

     ---------------------------------------- 20.2/20.2 MB 3.7 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 8.0.0
    Uninstalling pyarrow-8.0.0:
      Successfully uninstalled pyarrow-8.0.0


In [10]:
!pip install credentials

In [14]:
from snowflake.connector.pandas_tools import write_pandas
import snowflake.connector
import pandas as pd


In [16]:
import credentials

In [106]:
conn = snowflake.connector.connect(
    user = 'ashika',
    password = 'Cervello123#',
    account = 'bpwmwqd-bk67062',
    warehouse = 'compute_wh',
    database = 'RAW',
    schema = 'test',
)

In [107]:
cur = conn.cursor()

In [97]:
data = cur.execute('select * from users')

In [98]:
fdata = data.fetchall()

In [99]:
fdata

[(1, 'admin_user', 'admin@example.com', 'hashed_pw_1', 'admin'),
 (2, 'trainer_john', 'john_trainer@example.com', 'hashed_pw_2', 'trainer'),
 (3, 'student_anna', 'anna_student@example.com', 'hashed_pw_3', 'student'),
 (4, 'student_mark', 'mark_student@example.com', 'hashed_pw_4', 'student'),
 (5, 'trainer_emma', 'emma_trainer@example.com', 'hashed_pw_5', 'trainer')]

In [27]:
from pyspark.sql import SparkSession

spark = SparkSession.Builder().master('local[*]').appName("Pranav").getOrCreate()

In [28]:
columns = ['id','Name','email','role_id','role']

In [43]:
df = spark.createDataFrame(fdata, schema=columns)

In [44]:
df.show()

+---+------------+--------------------+-----------+-------+
| id|        Name|               email|    role_id|   role|
+---+------------+--------------------+-----------+-------+
|  1|  admin_user|   admin@example.com|hashed_pw_1|  admin|
|  2|trainer_john|john_trainer@exam...|hashed_pw_2|trainer|
|  3|student_anna|anna_student@exam...|hashed_pw_3|student|
|  4|student_mark|mark_student@exam...|hashed_pw_4|student|
|  5|trainer_emma|emma_trainer@exam...|hashed_pw_5|trainer|
+---+------------+--------------------+-----------+-------+



In [144]:
cur.execute("""
    SELECT table_name 
    FROM information_schema.tables
    WHERE table_schema = 'TEST' AND table_type = 'BASE TABLE'
""")

tables = cur.fetchall()
table_names = [table[0] for table in tables]

# Initialize an empty dictionary to store data from all tables
all_data = {}

# Fetch data from all tables
for table_name in table_names:
    cur.execute(f"SELECT * FROM {table_name} LIMIT 100")  # Limit the rows for simplicity
    data2 = cur.fetchall()
    df = pd.DataFrame(data2, columns=[col[0] for col in cur.description])
    all_data[table_name] = df


# Display the first few rows from all tables (for review)
for table, data2 in all_data.items():
    print(f"\nData from table: {table}")
    print(data2.head())


Data from table: CUSTOMERS
Empty DataFrame
Columns: [CUSTOMER_ID, FIRST_NAME, LAST_NAME, EMAIL, PHONE, ADDRESS, JOIN_DATE]
Index: []

Data from table: FEEDBACK
Empty DataFrame
Columns: [FEEDBACK_ID, USER_ID, PRODUCT_ID, RATING, COMMENTS, FEEDBACK_DATE]
Index: []

Data from table: INVENTORY
Empty DataFrame
Columns: [INVENTORY_ID, PRODUCT_ID, WAREHOUSE_LOCATION, QUANTITY_ON_HAND, LAST_RESTOCK_DATE]
Index: []

Data from table: ORDERS
Empty DataFrame
Columns: [ORDER_ID, USER_ID, ORDER_DATE, SHIPPING_ADDRESS, TOTAL_AMOUNT, ORDER_STATUS, PAYMENT_METHOD]
Index: []

Data from table: PAYMENTS
Empty DataFrame
Columns: [PAYMENT_ID, ORDER_ID, PAYMENT_DATE, PAYMENT_AMOUNT, PAYMENT_METHOD, PAYMENT_STATUS]
Index: []

Data from table: PRODUCTS
Empty DataFrame
Columns: [PRODUCT_ID, PRODUCT_NAME, CATEGORY, PRICE, STOCK_COUNT, RELEASE_DATE, IS_ACTIVE]
Index: []

Data from table: RETURNS
Empty DataFrame
Columns: [RETURN_ID, ORDER_ID, PRODUCT_ID, RETURN_DATE, REASON, REFUND_AMOUNT]
Index: []

Data from ta

In [104]:
cur.description

[ResultMetadata(name='SALE_ID', type_code=0, display_size=None, internal_size=None, precision=38, scale=0, is_nullable=False),
 ResultMetadata(name='PRODUCT_ID', type_code=0, display_size=None, internal_size=None, precision=38, scale=0, is_nullable=True),
 ResultMetadata(name='CUSTOMER_ID', type_code=0, display_size=None, internal_size=None, precision=38, scale=0, is_nullable=True),
 ResultMetadata(name='SALE_DATE', type_code=6, display_size=None, internal_size=None, precision=0, scale=9, is_nullable=True),
 ResultMetadata(name='QUANTITY_SOLD', type_code=0, display_size=None, internal_size=None, precision=38, scale=0, is_nullable=True),
 ResultMetadata(name='SALE_PRICE', type_code=0, display_size=None, internal_size=None, precision=10, scale=2, is_nullable=True),
 ResultMetadata(name='DISCOUNT', type_code=0, display_size=None, internal_size=None, precision=5, scale=2, is_nullable=True)]

In [105]:
all_data

{'CUSTOMERS': Empty DataFrame
 Columns: [CUSTOMER_ID, FIRST_NAME, LAST_NAME, EMAIL, PHONE, ADDRESS, JOIN_DATE]
 Index: [],
 'FEEDBACK': Empty DataFrame
 Columns: [FEEDBACK_ID, USER_ID, PRODUCT_ID, RATING, COMMENTS, FEEDBACK_DATE]
 Index: [],
 'INVENTORY': Empty DataFrame
 Columns: [INVENTORY_ID, PRODUCT_ID, WAREHOUSE_LOCATION, QUANTITY_ON_HAND, LAST_RESTOCK_DATE]
 Index: [],
 'ORDERS': Empty DataFrame
 Columns: [ORDER_ID, USER_ID, ORDER_DATE, SHIPPING_ADDRESS, TOTAL_AMOUNT, ORDER_STATUS, PAYMENT_METHOD]
 Index: [],
 'PAYMENTS': Empty DataFrame
 Columns: [PAYMENT_ID, ORDER_ID, PAYMENT_DATE, PAYMENT_AMOUNT, PAYMENT_METHOD, PAYMENT_STATUS]
 Index: [],
 'PRODUCTS': Empty DataFrame
 Columns: [PRODUCT_ID, PRODUCT_NAME, CATEGORY, PRICE, STOCK_COUNT, RELEASE_DATE, IS_ACTIVE]
 Index: [],
 'RETURNS': Empty DataFrame
 Columns: [RETURN_ID, ORDER_ID, PRODUCT_ID, RETURN_DATE, REASON, REFUND_AMOUNT]
 Index: [],
 'SHIPPING': Empty DataFrame
 Columns: [SHIPPING_ID, ORDER_ID, SHIPPING_METHOD, SHIPPING_D

In [114]:
cur.execute("""
    select get_ddl('SCHEMA','TEST');
""")

ff = cur.fetchall()

ff

[('create or replace schema TEST;\n\ncreate or replace TABLE CUSTOMERS (\n\tCUSTOMER_ID NUMBER(38,0) NOT NULL,\n\tFIRST_NAME VARCHAR(16777216),\n\tLAST_NAME VARCHAR(16777216),\n\tEMAIL VARCHAR(16777216),\n\tPHONE VARCHAR(16777216),\n\tADDRESS VARCHAR(16777216),\n\tJOIN_DATE TIMESTAMP_LTZ(9),\n\tprimary key (CUSTOMER_ID)\n);\ncreate or replace TABLE FEEDBACK (\n\tFEEDBACK_ID NUMBER(38,0) NOT NULL,\n\tUSER_ID NUMBER(38,0),\n\tPRODUCT_ID NUMBER(38,0),\n\tRATING NUMBER(38,0),\n\tCOMMENTS VARCHAR(16777216),\n\tFEEDBACK_DATE TIMESTAMP_LTZ(9),\n\tprimary key (FEEDBACK_ID)\n);\ncreate or replace TABLE INVENTORY (\n\tINVENTORY_ID NUMBER(38,0) NOT NULL,\n\tPRODUCT_ID NUMBER(38,0),\n\tWAREHOUSE_LOCATION VARCHAR(16777216),\n\tQUANTITY_ON_HAND NUMBER(38,0),\n\tLAST_RESTOCK_DATE TIMESTAMP_LTZ(9),\n\tprimary key (INVENTORY_ID)\n);\ncreate or replace TABLE ORDERS (\n\tORDER_ID NUMBER(38,0) NOT NULL,\n\tUSER_ID NUMBER(38,0),\n\tORDER_DATE TIMESTAMP_LTZ(9),\n\tSHIPPING_ADDRESS VARCHAR(16777216),\n\tTOTA

In [133]:

cur.execute("""
    SELECT 
    TABLE_NAME, 
    COLUMN_NAME, 
    DATA_TYPE, 
    IS_NULLABLE, 
    COLUMN_DEFAULT 
FROM 
    INFORMATION_SCHEMA.COLUMNS
WHERE 
    TABLE_SCHEMA = 'TEST'
    order by table_name
;
""")

metadata = cur.fetchall()

metadata

[('CUSTOMERS', 'CUSTOMER_ID', 'NUMBER', 'NO', None),
 ('CUSTOMERS', 'FIRST_NAME', 'TEXT', 'YES', None),
 ('CUSTOMERS', 'LAST_NAME', 'TEXT', 'YES', None),
 ('CUSTOMERS', 'EMAIL', 'TEXT', 'YES', None),
 ('CUSTOMERS', 'JOIN_DATE', 'TIMESTAMP_LTZ', 'YES', None),
 ('CUSTOMERS', 'PHONE', 'TEXT', 'YES', None),
 ('CUSTOMERS', 'ADDRESS', 'TEXT', 'YES', None),
 ('FEEDBACK', 'PRODUCT_ID', 'NUMBER', 'YES', None),
 ('FEEDBACK', 'USER_ID', 'NUMBER', 'YES', None),
 ('FEEDBACK', 'FEEDBACK_ID', 'NUMBER', 'NO', None),
 ('FEEDBACK', 'RATING', 'NUMBER', 'YES', None),
 ('FEEDBACK', 'FEEDBACK_DATE', 'TIMESTAMP_LTZ', 'YES', None),
 ('FEEDBACK', 'COMMENTS', 'TEXT', 'YES', None),
 ('INVENTORY', 'LAST_RESTOCK_DATE', 'TIMESTAMP_LTZ', 'YES', None),
 ('INVENTORY', 'PRODUCT_ID', 'NUMBER', 'YES', None),
 ('INVENTORY', 'QUANTITY_ON_HAND', 'NUMBER', 'YES', None),
 ('INVENTORY', 'INVENTORY_ID', 'NUMBER', 'NO', None),
 ('INVENTORY', 'WAREHOUSE_LOCATION', 'TEXT', 'YES', None),
 ('ORDERS', 'PAYMENT_METHOD', 'TEXT', 'YES',

In [134]:
import json

In [145]:
tables = {}

# Loop through each column data tuple
for table_name, column_name, data_type, is_nullable, default_value in metadata:
    # Initialize a new table in the dictionary if it doesn't exist
    if table_name not in tables:
        tables[table_name] = []
    
    # Add a new entry for the column in the table
    tables[table_name].append({
        "column_name": column_name,
        "data_type": data_type,
        "is_nullable": is_nullable  # Using None as placeholder value
    })

# Convert the tables dictionary to JSON format
json_data = json.dumps(tables, indent=4)

# Output the JSON data
print(json_data)

{
    "CUSTOMERS": [
        {
            "column_name": "CUSTOMER_ID",
            "data_type": "NUMBER",
            "is_nullable": "NO"
        },
        {
            "column_name": "FIRST_NAME",
            "data_type": "TEXT",
            "is_nullable": "YES"
        },
        {
            "column_name": "LAST_NAME",
            "data_type": "TEXT",
            "is_nullable": "YES"
        },
        {
            "column_name": "EMAIL",
            "data_type": "TEXT",
            "is_nullable": "YES"
        },
        {
            "column_name": "JOIN_DATE",
            "data_type": "TIMESTAMP_LTZ",
            "is_nullable": "YES"
        },
        {
            "column_name": "PHONE",
            "data_type": "TEXT",
            "is_nullable": "YES"
        },
        {
            "column_name": "ADDRESS",
            "data_type": "TEXT",
            "is_nullable": "YES"
        }
    ],
    "FEEDBACK": [
        {
            "column_name": "PRODUCT_ID",
            